# Optimizavimo metodų laboratorinis darbas
#### Elijas Dapšauskas TSf-17

In [2]:
if (!require(linprog)) install.packages('linprog')
library('linprog')
library('lpSolve')

Loading required package: linprog
Warning message:
"package 'linprog' was built under R version 3.5.3"Loading required package: lpSolve
Warning message:
"package 'lpSolve' was built under R version 3.5.3"

# 1.
### 1.1.
##### 1.1.1.

$x_1$ &mdash; užsakomos minutės radijuje<br>
$x_2$ &mdash; užsakomos minutės televizijoje

$$x_1+25x_2\rightarrow\max$$
$$\begin{cases} 
15x_1 + 300x_2 \leq 10000 \\
-x_1+2x_2 \leq 0 \\
x_1 \leq 400 \\
x_1,x_2 \geq 0
\end{cases}$$

In [3]:
radio.cost_per_min = 15
tv.cost_per_min = 300

optimizeAdBudget = function(available_budget) {
    cvec = c(1,25)
    Amat = rbind(c(radio.cost_per_min, tv.cost_per_min),
                 c(-1, 2),
                 c(1, 0))
    bvec= c(available_budget,
            0,
            400)
    solution = solveLP(cvec,bvec,Amat,maximum=T)$solution
    return(list(radio_mins=solution[1], tv_mins=solution[2]))
}

ad_budget = optimizeAdBudget(available_budget=10000)
print(paste('Televizijoje:', floor(ad_budget$tv_mins), 'min.'))
print(paste('Radijuje:', floor(ad_budget$radio_mins), 'min.'))

[1] "Televizijoje: 30 min."
[1] "Radijuje: 60 min."


##### 1.1.2.

In [4]:
monthly_radio_budget = radio.cost_per_min * floor(ad_budget$radio_mins)
print(paste(monthly_radio_budget, 'EUR'))

[1] "900 EUR"


##### 1.1.3.

In [5]:
ad_budget = optimizeAdBudget(available_budget=15000)
print(paste('Televizijoje:', floor(ad_budget$tv_mins), 'min.'))
print(paste('Radijuje:', floor(ad_budget$radio_mins), 'min.'))

[1] "Televizijoje: 45 min."
[1] "Radijuje: 90 min."


# 2.
### 2.1.
##### 2.1.1.

$x_i$ &mdash; "Ar stotis pastatoma $i$-ajame mieste?"<br>
$x_i \in \{0,1\}$ &mdash; čia $1$ reiškia "Taip" <br>
Apribojimai sudaromi taip, kad būtų bent vienas ($\geq 1$) miestas su pagalbos stotimi, iš kurio galima patekti į $i$-ąjį miestą per ne daugiau 15 min. Pavyzdžiui, į pirmąjį miestą šitaip galime patekti iš 1, 3, 4 miestų, todėl apribojimas $x_1+x_3+x_4 \geq 1$.

$$x_1+x_2+x_3+x_4+x_5+x_6\rightarrow\min$$
$$\begin{cases} 
x_1+x_3+x_4 \geq 1 \\
x_2+x_4+x_6 \geq 1 \\
x_1+x_3 \geq 1 \\
x_2+x_4 \geq 1 \\
x_1+x_5+x_6 \geq 1 \\
x_2+x_5+x_6 \geq 1
\end{cases}$$

##### 2.1.2.

Iš uždavinio sąlygos:

In [6]:
cities = 6
distance_limit = 15
distances = rbind(c(0,23,14,18,10,32),
                  c(23,0,24,13,22,11),
                  c(14,24,0,60,19,20),
                  c(18,13,60,0,55,17),
                  c(10,22,19,55,0,12),
                  c(32,11,20,17,12,0))

Išsprendžiame TPU:

In [7]:
cvec = rep(1, cities)
bvec = rep(1, cities)
Amat = rbind(c(radio.cost_per_min, tv.cost_per_min),
                 c(-1, 2),
                 c(1, 0))
solution = lp("min", cvec, Amat, '>=', bvec, int.vec=1:length(cvec), all.bin=T)$solution

print(paste("Mažiausias reikalingų stočių kiekis:", sum(solution)))
print(paste(c("Miestai kuriose statomos stotys:", which(solution>0)), collapse=" "))

Warning message in rbind(const.mat, const.dir.num, const.rhs):
"number of columns of result is not a multiple of vector length (arg 3)"

[1] "Mažiausias reikalingų stočių kiekis: 1"
[1] "Miestai kuriose statomos stotys: 2"


### 2.2.
##### 2.2.1.

Turime mašinų ilgių vektorių
$$c = (4, 4.5, 5, 4.1, 2.4, 5.2, 3.7, 3.5, 3.2, 4.5, 2.3, 3.3, 3.8, 4.6, 3)$$

Turime dvi gatvės puses, jas žymėsime "$0$" ir "$1$".<br>
$x_i$ reikšmė rodo, kurioje gatvės pusėje statoma i-oji mašina: $$x_i \in \{0,1\}$$.<br>

Sakykime, kad užstatyta gatvės pusė "$1$" visada yra ne trumpesnė nei užstatyta gatvės pusė "$0$".
Tai galime įgyvendinti apribojimu
$$\sum_i c_i x_i \geqslant \frac{1}{2}\sum_i c_i$$

Tikslo funkcija laikysime, kad ilgesnioji pusė būtų kiek įmanoma trumpesnė. Kadangi ilgesnioji pusė pas mus yra "$1$", tuomet minimizuosime tikslo funkciją: $$\min \sum_i c_i x_i$$

##### 2.2.2.

In [8]:
lengths = c(4, 4.5, 5, 4.1, 2.4, 5.2, 3.7, 3.5, 3.2, 4.5, 2.3, 3.3, 3.8, 4.6, 3)
cvec = lengths
bvec = sum(lengths) / 2
Amat = rbind(c(lengths))
solution = lp("min", cvec, Amat, '>=', bvec, int.vec=1:length(cvec), all.bin=T)$solution
print(paste(c("Mašinų numeriai vienoje gatvės pusėje:", which(solution==0)), collapse=" "))
print(paste("Užstatytas ilgis vienoje gatvės pusėje:", sum(lengths[which(solution==0)])))
print(paste(c("Mašinų numeriai antroje gatvės pusėje:", which(solution==1)), collapse=" "))
print(paste("Užstatytas ilgis antroje gatvės pusėje:", sum(lengths[which(solution==1)])))

[1] "Mašinų numeriai vienoje gatvės pusėje: 5 6 9 10 11 12 14 15"
[1] "Užstatytas ilgis vienoje gatvės pusėje: 28.5"
[1] "Mašinų numeriai antroje gatvės pusėje: 1 2 3 4 7 8 13"
[1] "Užstatytas ilgis antroje gatvės pusėje: 28.6"


### 2.3.
##### 2.3.1.

In [9]:
# Visos sumos skaičiuojamos 1 tūkst. eur matu
K = 1e4
profit = c(50, 100, 200, 300, 80, 200)
expenses = c(1000, 2000, 3000, 5000, 1500, 3500)

In [10]:
cvec = profit
Amat = rbind(expenses)
bvec = K

result = lp("max",cvec,Amat,'<=',bvec,int.vec=1:length(cvec),all.bin=T)
print(paste(c("Vykdomi projektai:", which(result$solution>0)), collapse=" "))

[1] "Vykdomi projektai: 2 3 4"


##### 2.3.2.

In [11]:
cvec = profit
Amat = rbind(expenses,
             c(1,0,0,0,0,0),
             c(0,1,0,0,0,0),
             c(0,0,1,0,0,0),
             c(0,0,0,1,0,0),
             c(0,0,0,0,1,0),
             c(0,0,0,0,0,1))
bvec = c(K,1,1,1,1,1,1)

result = lp("max",cvec,Amat,'<=',bvec,compute.sens=T)
print(paste(c("Vykdomi projektai:", which(result$solution>0)), collapse=" "))
print(paste(c("Projektų vykdymo koeficientai:", result$solution), collapse=" "))
print(paste(c("Jautrumo analizė. Nuo:", result$sens.coef.from), collapse=" "))
print(paste(c("Jautrumo analizė. Iki:", result$sens.coef.to), collapse=" "))

[1] "Vykdomi projektai: 3 4 6"
[1] "Projektų vykdymo koeficientai: 0 0 1 1 0 0.571428571428572"
[1] "Jautrumo analizė. Nuo: -1e+30 -1e+30 171.428571428571 285.714285714286 -1e+30 186.666666666667"
[1] "Jautrumo analizė. Iki: 57.1428571428571 114.285714285714 1e+30 1e+30 85.7142857142857 210"


### 2.4.

Kadangi kompanijos neturi apribojimų, kiek minimaliai ar maksimaliai skambučių galime pirkti, todėl visas minutes tiesiog pirksime pigiausiai paslaugą siūlančioje kompanijoje.

In [12]:
mins = 200
cost = c(16+.25*mins, 25+.21*mins, 18+.22*mins)

cat("I kompanija:", cost[1], "$\n")
cat("II kompanija:", cost[2], "$\n")
cat("III kompanija:", cost[3], "$\n\n")

cat("Pigiausia kompanija, kurioje galime pirkti visas minutes: ", which(min(cost)==cost), "\n")
cat("Paslaugų kaina: ", min(cost), "$")

I kompanija: 66 $
II kompanija: 67 $
III kompanija: 62 $

Pigiausia kompanija, kurioje galime pirkti visas minutes:  3 
Paslaugų kaina:  62 $

### 2.5.

Žymėsime tarpdurius skaičiais nuo 1 iki 8, eilės tvarka iš viršaus į apačią, tada iš kairės į dešinę. $x_i \in \{0,1\}$ žymėsime, ar pastatyti sargą į $i$-ąjį tarpdurį. Apribojimus sudarysime taip, kad bent viename kambariui priklausančiui tarpduryje būtų pastatytas sargas, pavyzdžiui viršutiniam kairiajam kambariui priklauso tarpduriai 1 ir 2, todėl turėsime apribojimą $x_1 + x_2 \geq 1$. Analogiškai sudarysime apribojimus ir likusiems kambariams. Kambariai surašomi tokia tvarka: iš viršaus į apačią, tada - iš kairės į dešinę.

In [13]:
door_count = 8
room_count = 8
bvec = rep(1,room_count)
cvec = rep(1,door_count)
Amat = rbind(c(1,1,0,0,0,0,0,0),
             c(1,0,1,0,0,0,0,0),
             c(0,1,0,1,0,0,0,0),
             c(0,0,0,1,1,0,0,0),
             c(0,0,1,0,1,1,0,0),
             c(0,0,0,0,0,0,1,0),
             c(0,0,0,0,0,0,1,1),
             c(0,0,0,0,0,1,0,1))
solution = lp("min",cvec,Amat,'>=',bvec,int.vec=1:length(cvec),all.bin=TRUE)$solution
print(paste("Mažiausias reikalingų sargų kiekis:", sum(solution)))
print(paste(c("Tarpduriai kuriuose statomi sargai:", which(solution>0)), collapse=" "))

[1] "Mažiausias reikalingų sargų kiekis: 4"
[1] "Tarpduriai kuriuose statomi sargai: 1 4 6 7"


### 2.6.
(Uždavinio atlikti nebūtina)

# 3.
### 3.1.

$x_1$ - Produktui 1, Operacijos 1 skirtos minutės<br>
$x_2$ - Produktui 2, Operacijos 1 skirtos minutės<br>
$x_3$ - Produktui 1, Operacijos 2 skirtos minutės<br>
$x_4$ - Produktui 2, Operacijos 2 skirtos minutės<br><br>

Turėsime šiuos apribojimus:

- Per dieną pagaminti bent 80 produkto 1 ir 60 produkto 2 (4 TPU apribojimai)
- Pabaigti pradėtus produktus (2 TPU apribojimai)
- Operacijai 1 ir Operacijai 2 skirti bent po 8h per dieną (2 TPU apribojimai)

In [32]:
p1o1 = 5 # Produktas 1, Operacija 1
p2o1 = 3
p1o2 = 6
p2o2 = 2
o1.mins.min = 8 * 60
o2.mins.min = 8 * 60
p1.quantity.min = 80
p2.quantity.min = 60
cvec = c(p1o1, p2o1, p1o2, p2o2)
Amat = rbind(c(1, 0, 0, 0),
             c(0, 1, 0, 0),
             c(0, 0, 1, 0),
             c(0, 0, 0, 1),
             c(p1o1, 0,    -p1o2, 0),
             c(0,    p2o1, 0,     -p2o2),
             c(1, 1, 0, 0),
             c(0, 0, 1, 1))
bvec = c(p1o1 * p1.quantity.min,
         p2o1 * p1.quantity.min,
         p1o2 * p1.quantity.min,
         p2o2 * p2.quantity.min,
         0,
         0,
         o1.mins.min,
         o2.mins.min)
costrvec = c('>=',
             '>=',
             '>=',
             '>=',
             '=',
             '=',
             '>=',
             '>=')
solution = lp("min",cvec,Amat,costrvec,bvec,int.vec=1:length(cvec))$solution
operation1.mins = solution[1] + solution[3]
operation2.mins = solution[2] + solution[4]
cat('Viršvalandžiai operacijai 1: ', operation1.mins - o1.mins.min, 'min\n')
cat('Viršvalandžiai operacijai 2: ', operation2.mins - o2.mins.min, 'min\n')

Viršvalandžiai operacijai 1:  576 min
Viršvalandžiai operacijai 2:  120 min
